In [20]:
import os
import shutil
import random
from pathlib import Path

# 定义路径和参数
DATA_DIR = "dataset/"
OUTPUT_DIR = "dataset/output/"
TRAIN_DIR = os.path.join(OUTPUT_DIR, "train")
TEST_DIR = os.path.join(OUTPUT_DIR, "test")
ANNOTATIONS_DIR = os.path.join(DATA_DIR, "annotations")
IMAGES_DIR = os.path.join(DATA_DIR, "images")
VOC_YAML_PATH ="voc.yaml"
DATASET_URL = "andrewmvd/face-mask-detection"
TRAIN_RATIO = 0.8

# 下载并解压 Kaggle 数据集
def download_and_extract(url, output_dir):
    try:
        import kagglehub
    except ImportError:
        print("❌ 未安装 kagglehub 库，请先安装：pip install kagglehub")
        return

    if os.path.exists(output_dir):
        print(f"✅ 数据集已存在，取消下载：{os.path.abspath(output_dir)}")
        return

    print("🚀 开始下载数据集...")
    url_path = kagglehub.dataset_download(url)
    shutil.copytree(url_path, output_dir)
    shutil.rmtree(url_path)
    print(f"✅ 数据集已成功下载到：{os.path.abspath(output_dir)}")

# 划分数据集并复制文件
def split_dataset(train_ratio):
    # 创建输出目录
    if os.path.exists(OUTPUT_DIR):
        print(f"✅ 输出目录已存在，取消划分：{os.path.abspath(OUTPUT_DIR)}")
    else:
        os.makedirs(TRAIN_DIR, exist_ok=True)
        os.makedirs(TEST_DIR, exist_ok=True)
        # 获取所有文件名
        annotation_files = list(Path(ANNOTATIONS_DIR).glob("*.xml"))
        image_files = list(Path(IMAGES_DIR).glob("*.png"))  # 假设图片格式为 .png

        # 确保注释文件和图片文件数量一致
        assert len(annotation_files) == len(image_files), "注释文件和图片文件数量不一致"

        # 打乱顺序
        combined = list(zip(annotation_files, image_files))
        random.shuffle(combined)
        annotation_files[:], image_files[:] = zip(*combined)

        # 划分训练集和测试集
        split_index = int(len(annotation_files) * train_ratio)
        train_annotations = annotation_files[:split_index]
        train_images = image_files[:split_index]
        test_annotations = annotation_files[split_index:]
        test_images = image_files[split_index:]

        # 复制文件到训练集和测试集目录
        def copy_files(file_list, target_dir):
            for file in file_list:
                shutil.copy(file, target_dir)

        copy_files(train_annotations, TRAIN_DIR)
        copy_files(train_images, TRAIN_DIR)
        copy_files(test_annotations, TEST_DIR)
        copy_files(test_images, TEST_DIR)

        print(f"✅ 数据集划分完成，训练集路径：{TRAIN_DIR}")
        print(f"✅ 数据集划分完成，测试集路径：{TEST_DIR}")
        with open(VOC_YAML_PATH, "w") as f:
            f.write(f"train: {os.path.abspath(TRAIN_DIR)}\n")
            f.write(f"val: {os.path.abspath(TEST_DIR)}\n")
            f.write("\n")
            f.write("nc: 2\n")  # 假设有两个类别，例如“戴口罩”和“未戴口罩”
            f.write("names: ['with_mask', 'without_mask']\n")  # 类别名称
        print(f"✅ voc.yaml 文件已生成：{VOC_YAML_PATH}")

# 生成 voc.yaml 文件
    

# 下载数据集
download_and_extract(DATASET_URL, DATA_DIR)# 划分数据集
split_dataset(TRAIN_RATIO)# 生成 voc.yaml 文件

✅ 数据集已存在，取消下载：c:\Users\Spark\Desktop\vision\ocr\dataset
✅ 数据集划分完成，训练集路径：dataset/output/train
✅ 数据集划分完成，测试集路径：dataset/output/test
✅ voc.yaml 文件已生成：voc.yaml


In [17]:
!pip install ultralytics
!nvidia-smi
from ultralytics import YOLO
device = 'cuda'  # 使用GPU训练,可选cuda或cpu

Sat Apr 19 16:49:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce MX250         WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   45C    P0             N/A / 5001W |       0MiB /   2048MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
model = YOLO("baseModel/yolov8n.pt")  # 使用预训练模型
model.train(
    data="voc.yaml",
    device=0 if device == "cuda" else "cpu",
    epochs=30, 
    batch=64,
    imgsz=640,
    optimizer="AdamW", 
    augment=True,
    lr0=0.001,               # 适当提高初始学习率
    lrf=0.01,                # 添加余弦退火最终学习率
    amp=True,               # 保持混合精度训练
    pretrained=True,        # 确保使用预训练权重
    save=True,  # 保存模型
    exist_ok=True,
    )  # 训练模型

New https://pypi.org/project/ultralytics/8.3.111 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.107  Python-3.9.0 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce MX250, 2048MiB)
engine\trainer: task=detect, mode=train, model=baseModel/yolov8n.pt, data=voc.yaml, epochs=30, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False,

100%|██████████| 5.35M/5.35M [00:01<00:00, 5.42MB/s]


AMP: checks passed 


train: Scanning C:\Users\Spark\Desktop\vision\ocr\dataset\output\train... 0 images, 682 backgrounds, 0 corrupt: 100%|██████████| 682/682 [00:01<00:00, 398.04it/s]

train: WARNING  No labels found in C:\Users\Spark\Desktop\vision\ocr\dataset\output\train.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


...

val: Scanning C:\Users\Spark\Desktop\vision\ocr\dataset\output\test... 0 images, 171 backgrounds, 0 corrupt: 100%|██████████| 171/171 [00:00<00:00, 360.13it/s]

val: WARNING  No labels found in C:\Users\Spark\Desktop\vision\ocr\dataset\output\test.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


val: New cache created: C:\Users\Spark\Desktop\vision\ocr\dataset\output\test.cache
WARNING  No labels found in C:\Users\Spark\Desktop\vision\ocr\dataset\output\test.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs\detect\train\labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/22 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacity of 2.00 GiB of which 0 bytes is free. Of the allocated memory 1.60 GiB is allocated by PyTorch, and 73.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)